# Steel Defects

Problem Statement: Based on the dataset provided by Severstal through Kaggle.

Objectives:
* practice computer vision machine learning models
* practice model deployment with AWS

In [1]:
import os

TO-DO LIST:
* data extraction
* data cleaning
* feature transformation
* target transformation
* baseline
* model selection
* hyperparameter optimization
* export model

# 1. Data Extraction

Data will be extracted using the kaggle API.

In [2]:
#!kaggle competitions download -c severstal-steel-defect-detection
#comment out if used in an environment with active repo


  0%|          | 0.00/1.57G [00:00<?, ?B/s]
  0%|          | 2.00M/1.57G [00:00<01:48, 15.5MB/s]
  0%|          | 5.00M/1.57G [00:00<01:15, 22.1MB/s]
  0%|          | 8.00M/1.57G [00:00<01:08, 24.3MB/s]
  1%|          | 11.0M/1.57G [00:00<01:24, 19.8MB/s]
  1%|          | 14.0M/1.57G [00:00<01:16, 22.0MB/s]
  1%|1         | 18.0M/1.57G [00:00<01:03, 26.3MB/s]
  1%|1         | 21.0M/1.57G [00:00<01:00, 27.6MB/s]
  2%|1         | 25.0M/1.57G [00:01<01:00, 27.4MB/s]
  2%|1         | 28.0M/1.57G [00:01<01:02, 26.3MB/s]
  2%|1         | 32.0M/1.57G [00:01<00:58, 28.1MB/s]
  2%|2         | 35.0M/1.57G [00:01<00:57, 28.9MB/s]
  2%|2         | 39.0M/1.57G [00:01<00:52, 31.0MB/s]
  3%|2         | 42.0M/1.57G [00:01<00:52, 31.0MB/s]
  3%|2         | 45.0M/1.57G [00:01<00:52, 31.1MB/s]
  3%|2         | 48.0M/1.57G [00:01<00:52, 31.2MB/s]
  3%|3         | 51.0M/1.57G [00:01<00:52, 31.2MB/s]
  3%|3         | 55.0M/1.57G [00:02<00:50, 32.2MB/s]
  4%|3         | 59.0M/1.57G [00:02<00:52, 30.7MB/s]
 